In [1]:
!pip install hdbscan==0.8.28
!pip install umap-learn==0.5.2
!pip install numpy==1.20
!pip install pymorphy2==0.9.1

In [2]:
import warnings
from pickle import dump, load, HIGHEST_PROTOCOL
from typing import Any
from datetime import (
    datetime, timezone
)

import nltk
import pymorphy2
import pandas as pd
import numpy as np
import hdbscan
import umap

from sklearn.feature_extraction.text import CountVectorizer

In [3]:
warnings.filterwarnings('ignore')
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erqups/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords

# 1. Служебные функции

In [29]:
class Pickler:
    @staticmethod
    def pickle(object_to_pickle: Any, file: str) -> None:
        with open(file, 'wb') as handle:
            dump(object_to_pickle, handle, protocol=HIGHEST_PROTOCOL)

    @staticmethod
    def unpickle(file: str) -> Any:
        with open(file, 'rb') as handle:
            return load(handle)
        

class ClassTfIdfTokensExtractor:
    def __init__(self, stop_words: list, ngram_range: tuple = (1, 1)):
        self._ngram_range = ngram_range
        self._stop_words = stop_words
    
    def _get_tf_idf(self, joined_clusters: np.ndarray, documents_count: int):
        count_vectorizer = CountVectorizer(
            ngram_range=self._ngram_range, stop_words=self._stop_words
        ).fit(
            joined_clusters
        )
        
        vectorized = count_vectorizer.transform(joined_clusters).toarray()
        vectorized_sum_hor = vectorized.sum(axis=1)
        vectorized_sum_vert = vectorized.sum(axis=0)
        
        tf = np.divide(vectorized.T, vectorized_sum_hor)
        idf = np.log(
            np.divide(
                documents_count, vectorized_sum_vert
            )
        ).reshape(-1, 1)
        
        tf_idf = np.multiply(tf, idf)

        return tf_idf, count_vectorizer

    def _get_topics_top_tokens(self, tf_idf, count_vectorizer, joined_clusters, topics, count):
        tokens = count_vectorizer.get_feature_names_out()
        labels = list(topics)
        
        tf_idf_transposed = tf_idf.T
        indices = tf_idf_transposed.argsort()[:, -count:]
        
        top_tokens = {
            label: [
                (
                    tokens[j], tf_idf_transposed[i][j]
                ) for j in indices[i]
            ][::-1] 
            for i, label in enumerate(labels)
        }
        
        return top_tokens
    
    def extract(self, joined_clusters: np.ndarray, topics, documents_count: int, count: int = 20):
        tf_idf, count_vectorizer = self._get_tf_idf(
            joined_clusters, documents_count
        )
        
        top_tokens = self._get_topics_top_tokens(
            tf_idf, count_vectorizer, joined_clusters, topics, count
        )
        
        return top_tokens
    
    
def get_tf_idf_tokens_by_subsample(russian_stop_words, morph_analyzer, tf_idf_extractor, subsample, topics):
    subsample_message = subsample["message"].fillna("").str.lower().str.replace(
        "[^a-zа-яё ]", " ", regex=True
    ).str.replace(
        "\s+", " ", regex=True
    ).values
    
    subsample_message_df = pd.DataFrame(
        subsample_message, columns=["message"]
    )
    
    subsample_message_df["topic"] = topics
    
    subsample_message_df_joint = subsample_message_df.groupby(
        ['topic'], as_index = False
    ).agg(
        {
            'message': ' '.join
        }
    )
    
    subsample_message_df_joint["message"] = subsample_message_df_joint["message"].apply(
        lambda value: " ".join(
            [
                morph_analyzer.parse(word)[0].normal_form 
                for word in value.split()
            ]
        )
    )
    
    subsample_message_df_joint["message"] = subsample_message_df_joint["message"].apply(
        lambda value: " ".join(
            [
                word for word in value.split()
                if not word in russian_stop_words
            ]
        )
    )
    
    extraction_result = tf_idf_extractor.extract(
        subsample_message_df_joint["message"].values, 
        subsample_message_df_joint["topic"].values, 
        len(subsample_message_df), 10
    )
    
    return extraction_result, subsample_message_df, subsample_message_df_joint

# 2. Константы

In [6]:
MESSAGES_EMBEDDINGS_PKL_FILE = "../data/processed/messages_to_modeling_embeddings.pkl"
MESSAGES_TO_MODELING_PARQUET = "../data/processed/messages_to_modeling.parquet"
# COMMENTS_EMBEDDINGS_PKL_FILE = "../data/processed/comments_to_modeling_embeddings.pkl"

START_DATE_LEFT = datetime(
    2022, 2, 24, 0, 0, 0, 
    tzinfo=timezone.utc
)

START_DATE_RIGHT = datetime(
    2022, 2, 25, 0, 0, 0, 
    tzinfo=timezone.utc
)

END_DATE_LEFT = datetime(
    2022, 3, 7, 0, 0, 0, 
    tzinfo=timezone.utc
)

END_DATE_RIGHT = datetime(
    2022, 3, 8, 0, 0, 0, 
    tzinfo=timezone.utc
)

# 3. Стоп слова

In [7]:
russian_stop_words = stopwords.words("russian") + [
    "хотел", "c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"
]

russian_stop_words = list(set(russian_stop_words))
len(russian_stop_words)

560

# 4. Готовим посты и эмбеддинги

In [8]:
messages_embeddings = Pickler.unpickle(MESSAGES_EMBEDDINGS_PKL_FILE)
messages_embeddings.shape

(3473, 768)

In [9]:
messages_to_topic = pd.read_parquet(
    MESSAGES_TO_MODELING_PARQUET
).reset_index(drop=True)

messages_to_topic.shape

(3473, 4)

## 4.1. Бьем на подвыборки

### 4.1.1. 24 февраля - проукраинские

In [10]:
ukr_24_feb = messages_to_topic[
    (
        messages_to_topic["channel_orientation"] == "ukr"
    )
    & (
        (messages_to_topic["date"] >= START_DATE_LEFT)
        & (messages_to_topic["date"] < START_DATE_RIGHT)
        
    )
]

ukr_24_feb = ukr_24_feb[
    ~ukr_24_feb["message"].duplicated()
]

ukr_24_feb.shape

(1426, 4)

### 4.1.2. 7 марта - проукраинские

In [11]:
ukr_7_mar = messages_to_topic[
    (
        messages_to_topic["channel_orientation"] == "ukr"
    )
    & (
        (messages_to_topic["date"] >= END_DATE_LEFT)
        & (messages_to_topic["date"] < END_DATE_RIGHT)
        
    )
]

ukr_7_mar = ukr_7_mar[
    ~ukr_7_mar["message"].duplicated()
]

ukr_7_mar.shape

(772, 4)

### 4.1.3. 24 февраля - пророссийские

In [12]:
rus_24_feb = messages_to_topic[
    (
        messages_to_topic["channel_orientation"] == "rus"
    )
    & (
        (messages_to_topic["date"] >= START_DATE_LEFT)
        & (messages_to_topic["date"] < START_DATE_RIGHT)
        
    )
]

rus_24_feb = rus_24_feb[
    ~rus_24_feb["message"].duplicated()
]

rus_24_feb.shape

(858, 4)

### 4.1.4. 7 марта - пророссийские

In [13]:
rus_7_mar = messages_to_topic[
    (
        messages_to_topic["channel_orientation"] == "rus"
    )
    & (
        (messages_to_topic["date"] >= END_DATE_LEFT)
        & (messages_to_topic["date"] < END_DATE_RIGHT)
        
    )
]

rus_7_mar = rus_7_mar[
    ~rus_7_mar["message"].duplicated()
]

rus_7_mar.shape

(289, 4)

# 5. Понижение размерности и topic modeling

## 5.1 24 февраля - проукраинские

In [124]:
%%time
ukr_24_feb_reducted = umap.UMAP(
    n_neighbors=15, n_components=5, 
    min_dist=0.1, metric='cosine'
).fit_transform(
    messages_embeddings[ukr_24_feb.index.values]
)

ukr_24_feb_reducted.shape

CPU times: user 6.1 s, sys: 154 ms, total: 6.25 s
Wall time: 5.4 s


(1426, 5)

In [148]:
ukr_24_feb_clusterizer = hdbscan.HDBSCAN(
    min_cluster_size=13,
    metric='l1',                    
    cluster_selection_method='leaf'
).fit(ukr_24_feb_reducted)

In [149]:
ukr_24_feb_vc = pd.Series(
    ukr_24_feb_clusterizer.labels_
).value_counts()

print(
    len(ukr_24_feb_vc)
)

ukr_24_feb_vc[:20]

16


-1     744
 12     95
 11     74
 8      72
 4      70
 6      66
 14     65
 2      56
 10     55
 0      35
 7      19
 13     19
 9      16
 1      14
 3      13
 5      13
dtype: int64

## 5.2. 7 марта - проукраинские

In [93]:
%%time
ukr_7_mar_reducted = umap.UMAP(
    n_neighbors=15, n_components=5, 
    min_dist=0.1, metric='cosine'
).fit_transform(
    messages_embeddings[ukr_7_mar.index.values]
)

ukr_7_mar_reducted.shape

CPU times: user 3.89 s, sys: 128 ms, total: 4.01 s
Wall time: 3.18 s


(772, 5)

In [114]:
ukr_7_mar_clusterizer = hdbscan.HDBSCAN(
    min_cluster_size=10,
    metric='l1',                    
    cluster_selection_method='leaf'
).fit(ukr_7_mar_reducted)

In [115]:
ukr_7_mar_vc = pd.Series(
    ukr_7_mar_clusterizer.labels_
).value_counts()

print(
    len(ukr_7_mar_vc)
)

ukr_7_mar_vc[:20]

16


-1     450
 7      52
 10     38
 6      35
 3      32
 4      22
 13     21
 0      18
 2      15
 5      15
 9      14
 11     14
 12     13
 14     13
 1      10
 8      10
dtype: int64

## 5.3. 24 февраля - пророссийские

In [20]:
%%time
rus_24_feb_reducted = umap.UMAP(
    n_neighbors=15, n_components=5, 
    min_dist=0.1, metric='cosine'
).fit_transform(
    messages_embeddings[rus_24_feb.index.values]
)

rus_24_feb_reducted.shape

CPU times: user 4.36 s, sys: 142 ms, total: 4.5 s
Wall time: 3.67 s


(858, 5)

In [21]:
rus_24_feb_clusterizer = hdbscan.HDBSCAN(
    min_cluster_size=15,
    metric='euclidean',                    
    cluster_selection_method='leaf'
).fit(rus_24_feb_reducted)

In [22]:
rus_24_feb_vc = pd.Series(
    rus_24_feb_clusterizer.labels_
).value_counts()

print(
    len(rus_24_feb_vc)
)

rus_24_feb_vc[:20]

12


-1     458
 8      63
 4      61
 3      60
 1      52
 0      36
 5      33
 2      20
 9      20
 6      19
 7      18
 10     18
dtype: int64

## 5.4. 7 марта - пророссийские

In [23]:
%%time
rus_7_mar_reducted = umap.UMAP(
    n_neighbors=15, n_components=5, 
    min_dist=0.1, metric='cosine'
).fit_transform(
    messages_embeddings[rus_7_mar.index.values]
)

rus_7_mar_reducted.shape

CPU times: user 2.11 s, sys: 9.29 ms, total: 2.11 s
Wall time: 2.12 s


(289, 5)

In [224]:
rus_7_mar_clusterizer = hdbscan.HDBSCAN(
    min_cluster_size=7,
    metric='euclidean',                    
    cluster_selection_method='leaf'
).fit(rus_7_mar_reducted)

In [225]:
rus_7_mar_vc = pd.Series(
    rus_7_mar_clusterizer.labels_
).value_counts()

print(
    len(rus_7_mar_vc)
)

rus_7_mar_vc[:20]

12


-1     131
 5      20
 4      18
 6      18
 3      17
 7      17
 0      17
 8      16
 1      12
 2       8
 9       8
 10      7
dtype: int64

# 6. Topic representation

In [214]:
morph_analyzer = pymorphy2.MorphAnalyzer(lang="ru")
tf_idf_extractor = ClassTfIdfTokensExtractor(
    stop_words=russian_stop_words, ngram_range=(1, 2)
)

## 6.1. 24 февраля - проукраинские

In [215]:
ukr_24_feb_extraction_result, ukr_24_feb_sub_df, ukr_24_feb_sub_df_joint = get_tf_idf_tokens_by_subsample(
    russian_stop_words, morph_analyzer, tf_idf_extractor, 
    ukr_24_feb, ukr_24_feb_clusterizer.labels_
)

In [216]:
for idx in ukr_24_feb_vc.index.values:
    if idx == -1:
        continue
        
    top_tokens = "_".join(
        [
            pair[0] for pair in ukr_24_feb_extraction_result[idx]
        ]
    )
    
    print(idx, top_tokens)

12 сообщать_взрыв_слышный_российский_аэропорт_военный_украина_бой_киев_украинский
11 https_com_www_https www_подписаться_facebook com_www facebook_подписываться_facebook_фейк
8 украина_путин_зеленский_бояться_президент_гражданин_оборона_армия_решение_враг
4 сирена_киев_луцк_днепр_харьков_харьков держаться_обращение всу_всу луцк_центр киев_видео
6 погибнуть_область_результат_удар_обстрел_склад_результат обстрел_раненый_ранить_одесский
14 санкция_заявить_британия_байден_акция_джонсон_актив_замораживать_великобритания_рубль
2 российский_ракета_танк_российский танк_техника_вертолёт_граница_крылатый ракета_крылатый_военный техника
10 рф_войско_танк_всу_остров_противник_район_уничтожить_удаться_украина
0 укр_расти_далее_област_ки_нформаца_йський_техн_пов_обстр
7 видео_фото_плакать расстреливать_плакать_расстреливать_расстреливать очевидец_оккупант харьков_уничтожить группа_очевидец уничтожить_пацан молодой
13 украина_путин_готовый_нато_европа_германия_честно_италия венгрия_кипр_венгрия кипр


1. Продвижение ВС РФ
2. Потери среди гражданских
3. Санкции, курс рубля
4. Действия ВСУ
5. Лидеры стран НАТО
6. Метро в качестве бомбоубежищ

In [188]:
ukr_24_feb_sub_df[
    ukr_24_feb_sub_df["topic"] == 5
].loc[401]["message"]

'сообщается что в киевское метро впускают всех без взимания оплаты а станции стали использоваться в качестве бомбоубежищ '

## 6.2. 24 февраля - пророссийские

In [189]:
rus_24_feb_extraction_result, rus_24_feb_sub_df, rus_24_feb_sub_df_joint = get_tf_idf_tokens_by_subsample(
    russian_stop_words, morph_analyzer, tf_idf_extractor, 
    rus_24_feb, rus_24_feb_clusterizer.labels_
)

In [190]:
for idx in rus_24_feb_vc.index.values:
    if idx == -1:
        continue
        
    top_tokens = "_".join(
        [
            pair[0] for pair in rus_24_feb_extraction_result[idx]
        ]
    )
    
    print(idx, top_tokens)

8 минобороны_минобороны рф_рф_военный_российский_военный инфраструктура_вывести_строй_вооружённый_инфраструктура
4 президент_путин_заявить_зеленский_украина_решение_оружие_начало_украинский_президент рф
3 войско_всу_тактический_группа_операция_российский_появиться_тактический группа_область_харьков
1 рейс_авиакомпания_отменить_полёт_аэропорт_ростов_ростов дон_дон_воздушный_март
0 рубль_доллар_биржа_индекс_торг_акция_нефть_курс_евро_мосбиржа
5 взрыв_сообщать_одесса_воинский_кадр_харьков_всу_украинский_украинский сми_киев
2 посольство киев_заседание_посольство_экстренный заседание_мид_созывать_закрывать_виза_перебой_эвакуировать
9 банк_втб_вводить_вводить санкция_сша_сша вводить_ограничение_санкция банк_санкция_минфин сша
6 выкладывать_выкладывать кадр_очевидец выкладывать_очевидец_кадр_издание_киев очевидец_киев_видео_гореть
7 киев_взрыв_сирена_серия взрыв_серия_взрыв киев_reuters_звук_киев reuters_слышать серия
10 санкция_великобритания_сша_жёсткий_санкция сша_джонсон_санкция рф_британ

1. Продвижение ВС РФ
2. Отмена авиарейсов
3. Мосбиржа, рубль, валюта
4. Санкции
5. Взрывы в Киеве
6. Эвакуация посольств из Киева

In [208]:
rus_24_feb_sub_df[
    rus_24_feb_sub_df["topic"] == 0
].sample(10)

,message,topic
313,газ в моменте превышал долларов за тысячу куб...,0
142,торги на московской бирже остановлены на неоп...,0
553,доллар выше рублей на бирже,0
155,курс доллара уже пробил историческую отметку и...,0
424,нефть марки brent выше долларов за баррель на...,0
241,акции втб падают на на бирже,0
128,курсы на покупку доллара и евро на сайте сберб...,0
397,индекс волатильности американского фондового ...,0
542,нефть марки brent выше долларов впервые с сен...,0
686,на токийской фондовой бирже началось резкое с...,0


## 6.3. 7 марта - проукраинские

In [209]:
ukr_7_mar_extraction_result, ukr_7_mar_sub_df, ukr_7_mar_sub_df_joint = get_tf_idf_tokens_by_subsample(
    russian_stop_words, morph_analyzer, tf_idf_extractor, 
    ukr_7_mar, ukr_7_mar_clusterizer.labels_
)

In [210]:
for idx in ukr_7_mar_vc.index.values:
    if idx == -1:
        continue
        
    top_tokens = "_".join(
        [
            pair[0] for pair in ukr_7_mar_extraction_result[idx]
        ]
    )
    
    print(idx, top_tokens)

7 николаев_харьков_последствие_видео_видео сума_любовь_харьков авиаудар_дон_сума_николаев харьков
10 удар_одесский_корабль_область_военный_погибнуть_николаев_ракета_штаб одесский_одесский областной
6 переговоры_делегация_раунд переговоры_раунд_переговоры украина_украина_украинский_заявка украина_вылететь_беларусь
3 мариуполь_житель_оккупант_митинг_слава_пленный_вновь митинг_всу самолёт_флаг украина_мелитополь вновь
4 николаевский_глава николаевский_область_николаев_николаевский область_оккупант_огонь_российский_утренний атака_атака российский
13 компания_россиянин_глобальный_российский_крупный_игра_нефть_stanley предсказывать_stanley_дефолт случиться
0 рубль_превысить_доллар_цена_евро_биржа_рост_газ_впервые история_газ европа
2 расти_укр_далее_студент_що_як_расти йська_йська_це_який
5 прийти_соловьёв_урок_урок история_электричество_шестиклассник_школьник_слава украина_домой_полиция
9 направление_противник_войско_наступление_продолжать_операция_потеря_мариуполь_киев_направление противни

1. Оборона украинских городов
2. Переговоры в Белоруссии
3. Антироссийские митинги
4. Уход компаний из России, дефолт
5. Рубль, валюты, энергоресурсы
6. Заявления стран НАТО и Китая

In [221]:
ukr_7_mar_sub_df[
    ukr_7_mar_sub_df["topic"] == 10
].sample(10)

,message,topic
632,в харькове в результате обстрела оккупантами с...,10
629,на аэродроме под херсоном уничтожены военных в...,10
571,в черном море военные украины нанесли удар по ...,10
189,сегодня с утра николаев обстреливали ракетные ...,10
489,под обуховым час назад наш летчик ас сбил крыл...,10
381,в запорожской области российские военные кадыр...,10
570,в макарове киевская область в результате авиау...,10
753,появилось видео обстрела николаева вероятно и...,10
213,одесская область подверглась удару с моря в ра...,10
635,только что был нанесен ракетный удар по морско...,10


## 6.4. 7 марта - пророссийские

In [226]:
rus_7_mar_extraction_result, rus_7_mar_sub_df, rus_7_mar_sub_df_joint = get_tf_idf_tokens_by_subsample(
    russian_stop_words, morph_analyzer, tf_idf_extractor, 
    rus_7_mar, rus_7_mar_clusterizer.labels_
)

In [227]:
for idx in rus_7_mar_vc.index.values:
    if idx == -1:
        continue
        
    top_tokens = "_".join(
        [
            pair[0] for pair in rus_7_mar_extraction_result[idx]
        ]
    )
    
    print(idx, top_tokens)

5 житель_власть_появиться_местный_всу_видео_автомобиль_посольство_студент_эвакуироваться
4 ядерный_местный_житель_помощь_гуманитарный помощь_зеленский_куляк_гимнаст_российский_конвой
6 продажа_приостанавливать_онлайн продажа_онлайн_магазин_компания_приостановить_sephora_прекращать_sephora закрывать
3 переговоры_рф украина_раунд_раунд переговоры_мединский_переговоры рф_переговоры украина_делегация_рф_украинский делегация
7 правительство_министр_энергия_товар_пресс служба_недружественный_российский_служба_пресс_ввести
0 кубометр_тысяча_тысяча кубометр_рубль_газ_доллар_газ европа_европа_цена газ_цена
8 минобороны_мариуполь_гуманитарный_гуманитарный коридор_украина минобороны_минобороны рф_коридор_населить_населить пункт_харьков
1 санкция_импорт_поставка_ес_нефть рф_европа_нефть_газпром_бойкот_импорт нефть
2 делегация_переговоры_российский делегация_глава_дипломатический_переговоры дипломатический_ситуация_подоляк_коридор_гуманитарный коридор
9 нефтебаза_взрыв_луганский_лнр_луганск_обстрел

1. Гуманитарная помощь, коридоры
2. Уход компаний из России, приостановка
3. Переговоры в Белоруссии
4. Цена на энергоресурсы в Европе
5. Санкции, недружественные страны
6. Взрыв нефтебазы в ЛНР

In [238]:
rus_7_mar_sub_df[
    rus_7_mar_sub_df["topic"] == 9
]

,message,topic
3,около киева прогремел мощный взрыв по неподтве...,9
21,в николаеве возобновились обстрелы в небе видн...,9
24,мирная жительница погибла в результате артилл...,9
67,появились кадры тушения пожара на луганской не...,9
70,причина взрыва на нефтебазе в луганске попадан...,9
74,в луганске прогремел сильный взрыв по предвари...,9
225,вооружённые силы украины предположительно нане...,9
287,в луганске взрыв предположительно горит нефтеб...,9
